In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [2]:
import pandas as pd

In [3]:
options = webdriver.ChromeOptions()

In [4]:
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

In [5]:
url = "https://pocket.pokemongohub.net/en"

In [6]:
driver.get(url)

In [7]:
df = pd.DataFrame(columns=['name', 'rarity', 'link'])

In [8]:
set_list = WebDriverWait(driver,10).until(
  EC.presence_of_element_located((By.XPATH, "/html/body/main/div/section/section[1]/ul"))
)

sets = [li.find_element(By.TAG_NAME, 'a').get_attribute('href') for li in set_list.find_elements(By.XPATH, "./li")]

In [9]:
for set in sets:
  driver.get(set)

  ul_element = WebDriverWait(driver,10).until(
    EC.presence_of_element_located((By.XPATH, "/html/body/main/div/div[2]/div[2]/ul"))
  )
  
  li_elements = [li.find_element(By.TAG_NAME, 'a').get_attribute('href') for li in ul_element.find_elements(By.XPATH, "./li")]

  for link in li_elements:
    driver.get(link)
    name = WebDriverWait(driver, 10).until(
      EC.presence_of_element_located((By.XPATH, "/html/body/main/div/section[1]/header/h1"))
    )
    rarity = WebDriverWait(driver,10).until(
      EC.presence_of_element_located((By.XPATH, "//th[text()='Rarity description']/following-sibling::td"))
    )
    link = WebDriverWait(driver, 10).until(
      EC.presence_of_element_located((By.XPATH, "/html/body/main/div/section[1]/section[1]/div/img[1]"))
    )
    df.loc[len(df)] = {
      'name': name.text,
      'rarity': rarity.text,
      'link': link.get_attribute('src')
    }

In [10]:
driver.quit()

In [11]:
df.to_csv("data/cards.csv")